In [ ]:
import torch
import numpy as np
import pandas as pd
import time
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, DistilBertConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


train_df = pd.read_csv("ISEAR_train.csv")
test_df = pd.read_csv("ISEAR_test.csv")


unique_emotions = sorted(train_df["emotion"].unique())
emotion2label = {emotion: idx for idx, emotion in enumerate(unique_emotions)}
label2emotion = {idx: emotion for emotion, idx in emotion2label.items()}

train_df["label"] = train_df["emotion"].map(emotion2label)
test_df["label"] = test_df["emotion"].map(emotion2label)


train_dataset = Dataset.from_pandas(train_df[["text", "label"]])
test_dataset = Dataset.from_pandas(test_df[["text", "label"]])


model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


num_labels = len(emotion2label)
config = DistilBertConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    dropout=0.3,  # Dropout pre skryté vrstvy
    attention_dropout=0.3  # Dropout pre pozornosť
)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)


training_args = TrainingArguments(
    output_dir="./distilbert_finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.1,  # Zvýšenie weight decay pre silnejšiu regularizáciu
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,  # Načítanie najlepšieho modelu
    metric_for_best_model="eval_loss",  # Metrika pre early stopping
    greater_is_better=False,  # Nižšia validačná strata je lepšia
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Pridanie early stopping
)


start_time = time.time()

trainer.train()

end_time = time.time()
training_time = end_time - start_time

print(f" Training completed!  Training time: {training_time / 60:.2f} minutes")

model.save_pretrained("./distilbert_finetuned")
tokenizer.save_pretrained("./distilbert_finetuned")

results = trainer.evaluate()
print(" DistilBERT Evaluation Results:")
print(results)

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

Map:   0%|          | 0/1532 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


⏳ Starting DistilBERT fine-tuning on ISEAR dataset...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.229500,1.106296,0.624021,0.644701,0.624021,0.621497
2,0.891800,0.933663,0.664491,0.674422,0.664491,0.662275
3,0.876000,0.946623,0.678198,0.689967,0.678198,0.678089
4,0.685300,0.929255,0.693864,0.694598,0.693864,0.691716
5,0.809000,0.908010,0.697781,0.704625,0.697781,0.700417
6,0.507500,0.929995,0.707572,0.709312,0.707572,0.707983
7,0.479000,0.936001,0.706266,0.711072,0.706266,0.707525
8,0.440800,0.964762,0.706919,0.707996,0.706919,0.707044


✅ Training completed! 🕒 Training time: 3.18 minutes


📊 DistilBERT Evaluation Results:
{'eval_loss': 0.9080100059509277, 'eval_accuracy': 0.6977806788511749, 'eval_precision': 0.7046251446015811, 'eval_recall': 0.6977806788511749, 'eval_f1': 0.7004167240953819, 'eval_runtime': 1.3919, 'eval_samples_per_second': 1100.644, 'eval_steps_per_second': 68.97, 'epoch': 8.0}
